# Análisis de los resultados en la búsqueda de hiperparámetros con búsqueda aleatoria

En el notebook anterior, vimos cómo implementar una búsqueda aleatoria para la configuración de hiperparámetros de un clasificador `HistGradientBoostingClassifier` y se entrenó el modelo con el dataset `adult_census`.

En la práctica, suele utiliarse un gran número de iteraciones para la búsqueda aleatoria de hiperparámetros. Con el objetivo de evitar un alto costo computacional y poder hacer un análisis decente, cargaremos los resultados obtenidos de una búsqueda similar con 500 iteraciones.

In [ ]:
import pandas as pd

cv_results = pd.read_csv("randomized_search_results.csv", index_col=0)
cv_results

Definimos un función para remover los prefijos en los hiperparámetros.

In [ ]:
def shorten_param(param_name):
    if "__" in param_name:
        return param_name.rsplit("__", 1)[1]
    return param_name


cv_results = cv_results.rename(shorten_param, axis=1)
cv_results

Debido a que fueron más de dos hiperparámetros los que se configuraron en la búsqueda aleatoria, no podemos visualizar los resultados utilizando un mapa de calor. Podríamos utilizar un gráfico de dispersión para analizar pares de hiperparámetros, pero una proyección bidimensional en un problema multidimensional podría ocacionar una mala interpretación de los resultados. Veamos el ejemplo a continuación:

In [ ]:
import seaborn as sns
import numpy as np

df = pd.DataFrame(
    {
        "max_leaf_nodes": cv_results["max_leaf_nodes"],
        "learning_rate": cv_results["learning_rate"],
        "score_bin": pd.cut(
            cv_results["mean_test_score"], bins=np.linspace(0.5, 1.0, 6)
        ),
    }
)
sns.set_palette("YlGnBu_r")
ax = sns.scatterplot(
    data=df,
    x="max_leaf_nodes",
    y="learning_rate",
    hue="score_bin",
    s=50,
    color="k",
    edgecolor=None,
)
ax.set_xscale("log")
ax.set_yscale("log")

_ = ax.legend(
    title="mean_test_score", loc="center left", bbox_to_anchor=(1, 0.5)
)

En el gráfico anterior podemos ver que los mejores valores de desempeño están ubicados en una franja donde el `learning_rate` varía entre 0.01 y 1.0. Sin embargo, no tenemos control en cómo los otros hiperparámetros reaccionan a estos valores.

Para visualizar todos los hiperparámtros, podemos utilizar un gráfico de coordenadas paralelas (parallel coordinates plot).

In [ ]:
import numpy as np
import plotly.express as px

fig = px.parallel_coordinates(
    cv_results.rename(shorten_param, axis=1).apply(
        {
            "learning_rate": np.log10,
            "max_leaf_nodes": np.log2,
            "max_bins": np.log2,
            "min_samples_leaf": np.log10,
            "l2_regularization": np.log10,
            "mean_test_score": lambda x: x,
        }
    ),
    color="mean_test_score",
    color_continuous_scale=px.colors.sequential.Viridis,
)
fig.show()

**Nota**

Transformamos la mayoría de los valores de los ejes aplicando log10 o log2 para ampliar los rangos activos y mejorar la legibilidad de la gráfica.

La gráfica de coordenadas paralelas muestra los valores de los hiperparámetros en diferentes columnas, mientras que la métrica de desempeño se representa mediante un código de color. De este modo, podemos inspeccionar rápidamente si existe un rango de hiperparámetros que funciona bien o no.

Es posible **seleccionar un rango de resultados haciendo clic y manteniendo presionado en cualquier eje** de la gráfica de coordenadas paralelas. Luego puedes deslizar (mover) esa selección y cruzarla con otra selección para ver sus intersecciones. Puedes deshacer una selección haciendo clic nuevamente en el mismo eje.

En particular, para esta búsqueda de hiperparámetros, es interesante confirmar que las **líneas amarillas** (los modelos con mejor rendimiento) alcanzan valores intermedios en `learning rate`, es decir, valores entre −2 y 0 en la escala del eje, lo cual corresponde a tasas de aprendizaje entre 0.01 y 1.0 una vez invertida la transformación log10.

También podemos observar que no es posible seleccionar los modelos de mejor rendimiento eligiendo líneas en el eje `max_bins` con valores entre 1 y 3. Pero a medida que incrementamos el valor de este hiperparámetro, el rendimiento del modelo tiende a mejorar.

Los otros hiperparámetros no son muy sensibles. Podemos verificar que, si seleccionamos en el eje `learning_rate` valores entre −1.5 y −0.5 y en el eje `max_bins` valores entre 5 y 8, siempre seleccionamos modelos de alto rendimiento, sin importar los valores de los otros hiperparámetros.


## Resumen
En este notebook vimos cómo explorar interactivamente los resultados de una búsqueda aleatoria extensa con múltiples hiperparámetros que interactúan entre sí.

En particular, observamos que:

* Algunos hiperparámetros tienen muy poco impacto en la puntuación de validación cruzada.
* Otros deben ajustarse dentro de un rango específico para obtener modelos con buena precisión predictiva.
